In [1]:
# Import libraries
import requests
import re # regular expressions
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import unidecode
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
# local variables
subCategorListTitle  = "j-menu-item"
CHROME_DRIVER_PATH = r'C:\Users\User\Downloads\chromedriver_win32\chromedriver.exe'
URL = 'https://www.wildberries.ru/catalog/muzhchinam/odezhda/bryuki-i-shorty'


driver = webdriver.Chrome(CHROME_DRIVER_PATH)
driver.get(URL)

content = driver.page_source
soup = BeautifulSoup(content)

prices = []
brands = []
categories = []

data = {
        "Categories" : categories,
        "Brand" : brands,
        "Price" : prices
}

df = pd.DataFrame(data)

In [3]:
#To wait till page loads

def waitForPageLoads():
    driver.get(URL)
    delay = 3
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'lower-price')))
        print("Page is ready!")
    except TimeoutException:
        print("Loading took too much time!")
        
waitForPageLoads()        


Page is ready!


In [4]:
def searchAllItemsOfCurrentPage():
    class_title = 'j-card-item'
    prices = []
    brands = []
    categories = []
    res = []
    sizes_available = []
    content = driver.page_source
    soup = BeautifulSoup(content)
    for block in soup.findAll('div', attrs={'class':class_title}):
        class_title = 'j-card-item'

        # Get price of clothes
        priceString = "0"
        priceBlock = block.find('ins', attrs={'class': 'lower-price'})
        if(priceBlock != None):
            priceString = block.find('ins', attrs={'class': 'lower-price'}).text
        else:
            priceString = block.find('span', attrs={'class': 'lower-price'}).text
        price = unidecode.unidecode(priceString.strip())
        prices.append(price)
        # Get brand of clothes
        brandString = block.find('strong', attrs={'class': 'brand-name'}).text
        brand = unidecode.unidecode(brandString.strip())
        brands.append(brand)

        # Get category of clothes
        categoryString = block.find('span', attrs={'class': 'goods-name'}).text
        category = unidecode.unidecode(categoryString.strip())
        categories.append(category)
        sizes = []
    res.append(categories)
    res.append(brands)
    res.append(prices)
    addDataToDF(res, df)
        
        # todo
        # Get sizes of clothes
        
    #    for sizes_block in block.find('span', attrs = {'class': 'product-card__sizes'}):
    #        sizeString = sizes_block.find('a', attrs={'class': 'j-size'}).text
    #        size = unidecode.unidecode(sizeString.strip())
    #        sizes.append(size)
    #        print(sizes)
    #     sizes_available.append(sizes)

In [5]:
def addDataToDF(list, df):
    count = 0
    df2 = pd.DataFrame(data=list).T
    df2.columns = ["Categories", "Brand", "Price"]

    df = pd.concat([df,df2], axis = 0)
    
    df.to_excel('data.xlsx',index=False)
    print(df)

In [6]:
#buttons
burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
forWomenButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--306")
forMenButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--566")
shoesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--629")
forBabiesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--115")
accessoriesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--1")
allButtons = [forMenButton, forWomenButton, shoesButton, forBabiesButton, accessoriesButton]

searchAllItemsOfCurrentPage()

#press burger button and category shoes
ActionChains(driver).click(burgerButton).pause(1).click(shoesButton).perform()

#get again the content after pressing burger button
content = driver.page_source
soup = BeautifulSoup(content)

# block = soup.find('ul', attrs={'class':subCategorListTitle})
# print(block)

catalogList = driver.find_element(By.CLASS_NAME, "menu-catalog__list-2")
items = catalogList.find_elements_by_tag_name("li")


for block in items:
    print(block)
    ActionChains(driver).click(block).pause(5).perform()
    searchAllItemsOfCurrentPage()
#     ActionChains(driver).click(burgerButton).pause(1).click(shoesButton).perform()


                                            Categories  \
0            Briuki muzhskie, shtany muzhskie, khlopok   
1             briuki shtany muzhskie chernye dzhoggery   
2         Briuki muzhskie sportivnye, shtany dzhoggery   
3                   Briuki muzhskie sportivnye osennie   
4            Briuki muzhskie (dzhoggery, shtany kargo)   
..                                                 ...   
100            Chernye muzhskie shtany dzhoggery kargo   
101             Briuki priamye muzhskie, klassicheskie   
102         Briuki muzhskie uteplennye zimnie nachesom   
103  Briuki priamye sportivnye domashnie Triko poda...   
104      Briuki klassicheskie muzhskie briuki muzhskie   

                         Brand    Price  
0                         IMHO  1 258 R  
1                  Fashion.man  2 040 R  
2                 Barouz Jeans  1 792 R  
3                     DreamFit  1 320 R  
4    Hooligan STORE multibrand  3 330 R  
..                         ...      ...  
100      

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=106.0.5249.119)
